# Display The Data


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from lib.utils.loader import (
    load_normal,
    load_color,
    load_depth,
    load_mask,
)

image_idx = 0
data_dir = Path("/home/borth/GuidedResearch/data/dphm_christoph_mouthmove")

color = load_color(data_dir, image_idx, return_tensor="img")
display(color)

depth = load_depth(data_dir, image_idx, return_tensor="np", smooth=False)
plt.imshow(depth[400:600, 920:1100])
plt.show()

mask = load_mask(data_dir, image_idx, return_tensor="np")
plt.imshow(mask)
plt.show()

normal = load_normal(data_dir, image_idx, return_tensor="np", smooth=True)
normal[:, :, 1] = -normal[:, :, 1]
normal[:, :, 2] = -normal[:, :, 2]
plt.imshow(((((normal + 1) / 2) * 255).astype(np.uint8)))
plt.show()

In [ ]:
depth.min()

In [ ]:
from lib.renderer.camera import depth2camera
from lib.utils.loader import load_intrinsics, load_depth
import matplotlib.pyplot as plt
import numpy as np
from lib.utils.visualize import load_pcd
import open3d as o3d

image_idx = 0
data_dir = "/home/borth/GuidedResearch/data/dphm_christoph_mouthmove"

scale = 0.25
K = load_intrinsics(
    data_dir=data_dir,
    return_tensor="pt",
    scale=scale,
)
depth = load_depth(data_dir, image_idx, return_tensor="pt", smooth=True)
point, mask = depth2camera(depth, K, scale)
pcd = load_pcd(point.reshape(-1, 3).detach().cpu().numpy())
o3d.visualization.draw_plotly([pcd])

In [ ]:
import torch

height = 1080
width = 1920

near = 1.0
far = 100
fx = 914.4150
fy = 914.0300
cx = 959.5980
cy = 547.2020
A = near + far
B = near * far

# l = 0
# r = width
# b = height
# t = 0
l = -width/2
r = width/2
b = -height/2
t = height

tx = -(r + l) / (r - l)
ty = -(t + b) / (t - b)
tz = -(far + near) / (far - near)

Persp = torch.tensor(
    [
        [fx, 0.0, -cx, 0.0],
        [0.0, fy, -cy, 0.0],
        [0.0, 0.0, A, B],
        [0.0, 0.0, -1.0, 0.0],
    ]
)

NDC = torch.tensor(
    [
        [2 / (r - l), 0.0, 0.0, tx],
        [0.0, 2 / (t - b), 0.0, ty],
        [0.0, 0.0, -2 / (far - near), tz],
        [0.0, 0.0, 0.0, 1.0],
    ]
)

Proj = NDC @ Persp
print(Proj)

In [ ]:
normal.shape

# Pipnet Landmarks


In [ ]:
from lib.utils.loader import (
    load_pipnet_image,
    load_pipnet_landmark_2d,
    load_pipnet_landmark_3d,
)

pipnet_landmarks_2d = load_pipnet_landmark_2d(data_dir, idx=image_idx)

print(f"{pipnet_landmarks_2d.shape=}")
print(pipnet_landmarks_2d[:5, :])

pipnet_landmarks_3d = load_pipnet_landmark_3d(data_dir, idx=image_idx)
print(f"{pipnet_landmarks_3d.shape=}")
print(pipnet_landmarks_3d[:5, :])
print("depth", pipnet_landmarks_3d[:, 2])
plt.hist(pipnet_landmarks_3d[:, 2])
plt.show()

pipnet_image = load_pipnet_image(data_dir, idx=image_idx)
display(pipnet_image)

# Medipipe Landmarks


In [ ]:
from lib.utils.loader import (
    load_mediapipe_image,
    load_mediapipe_landmark_2d,
    load_mediapipe_landmark_3d,
)

mediapipe_landmarks_2d = load_mediapipe_landmark_2d(data_dir, idx=image_idx)

print(f"{mediapipe_landmarks_2d.shape=}")
print(mediapipe_landmarks_2d[:5, :])

mediapipe_landmarks_3d = load_mediapipe_landmark_3d(data_dir, idx=image_idx)
print(f"{mediapipe_landmarks_3d.shape=}")
print(mediapipe_landmarks_3d[:5, :])
print("depth", mediapipe_landmarks_3d[:50, 2])
plt.hist(mediapipe_landmarks_3d[:, 2])
plt.show()

mediapip_image = load_mediapipe_image(data_dir, idx=image_idx)
display(mediapip_image)

In [ ]:
from lib.model.utils import load_static_landmark_embedding

flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame_landmarks = load_static_landmark_embedding(flame_dir)
flame_landmarks["landmark_indices"]

# Depth

From https://cvg.cit.tum.de/data/datasets/rgbd-dataset/file_formats

The color and depth images are already pre-registered using the OpenNI driver from PrimeSense, i.e., the pixels in the color and depth images correspond already 1:1.

The depth images are scaled by a factor of 1000, i.e., a pixel value of 1000 in the depth image corresponds to a distance of 1 meter from the camera. A pixel value of 0 means missing value/no data.


In [ ]:
from lib.model.flame import FLAME
from lib.renderer.renderer import Renderer
import torch
import matplotlib.pyplot as plt
from lib.utils.mesh import vertex_normals

flame_dir = "/home/borth/GuidedResearch/checkpoints/flame2023"
data_dir = "/home/borth/GuidedResearch/data/dphm_christoph_mouthmove"
scale = 1.0
flame = FLAME(
    flame_dir=flame_dir,
    data_dir=data_dir,
    vertices_mask="full",
).to("cuda")
flame.init_params(
    global_pose=[0.0, 0, 0],
    transl=[0.0, 0.0, -0.5],
)
vertices, landmarks = flame()
renderer = flame.renderer()

# mesh = trimesh.Trimesh(vertices[0].detach().cpu().numpy(), faces=flame.faces.detach().cpu().numpy())
# vn = torch.tensor(mesh.vertex_normals).unsqueeze(0).to(vertices.device)
faces = flame.faces[:, [0, 2, 1]]
vn = vertex_normals(vertices=vertices, faces=faces)
normal, mask = renderer.render(vertices, faces, vn)
# normal, mask = renderer.render(vertices,flame.masked_faces(vertices), vn)
normal_image = renderer.normal_to_normal_image(normal, mask)

In [ ]:
from lib.utils.visualize import load_pcd
import open3d as o3d

pcd = load_pcd(vertices.detach().cpu().numpy()[0])
o3d.visualization.draw_plotly([pcd])

In [ ]:
from lib.utils.loader import (
    load_mediapipe_landmark_2d,
    load_mediapipe_landmark_3d,
    load_normal,
)
from lib.renderer.camera import camera2pixel

face_idx = 0
# face_idx = 110
# normal = load_normal(data_dir, image_idx, return_tensor="np")
# normal = (((normal + 1) / 2) * 255).astype(np.uint8)
K = load_intrinsics(data_dir=data_dir, return_tensor="pt", scale=1.0)
# face = vertices[:, flame.faces][0][face_idx]
# pixel = camera2pixel(face, K["fx"], K["fy"], K["cx"], K["cy"])
# pixel = camera2pixel(landmarks[0], K["fx"], K["fy"], K["cx"], K["cy"])

points = landmarks[0] 
# xc = points[:, 0]
# yc = points[:, 1]
# zc = points[:, 2]

# us = cx + fx * (xc / zc)
# vs = cy + fy * (yc / zc)

pixels = (K.to("cuda") @ points.T).T

plt.imshow(color)

# draw all of the lm on the screen
# x, y, _ = pixel[0]

for p in pixels:
    plt.scatter(int(p[0]), int(p[1]), c="red", s=2)  # Drawing a red point for each landmark
# plt.scatter(int(pixel[0,0]), int(pixel[0,1]), c="red", s=2)  # Drawing a red point for each landmark
# plt.scatter(int(pixel[1,0]), int(pixel[1,1]), c="blue", s=2)  # Drawing a red point for each landmark
# plt.scatter(int(pixel[2,0]), int(pixel[2,1]), c="green", s=2)  # Drawing a red point for each landmark
plt.show()

In [ ]:
K

In [ ]:
landmarks = landmarks[0]
landmarks[:, 2] = -landmarks[:, 2]

In [ ]:
landmarks

In [ ]:
normal.shape

In [ ]:
normal[0][int(pixel[0,1]), int(pixel[0,0])]

In [ ]:
face

In [ ]:
face

In [ ]:
from lib.model.utils import load_static_landmark_embedding

flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame_landmarks = load_static_landmark_embedding(flame_dir)
print(flame_landmarks["landmark_indices"][1])
print(flame_landmarks["lmk_face_idx"][1])

In [ ]:
lm_idx = 0

lm3d = load_pipnet_landmark_3d(data_dir, idx=image_idx)[lm_idx]
print(lm3d)

lm2d = load_pipnet_landmark_2d(data_dir, idx=image_idx)[lm_idx]
x, y = lm2d.astype(int)
print(lm2d)

depth = load_depth_masked(data_dir, image_idx, return_tensor="np", depth_factor=1000)
print(depth[y, x])
plt.imshow(depth)

x, y = lm2d.astype(int)
plt.scatter(x, y, c="red", s=10)  # Drawing a red point for each landmark
plt.show()

In [ ]:
import numpy as np
from lib.renderer.camera import load_intrinsics, pixel2camera
from lib.utils.loader import load_depth_masked
import torch
from pathlib import Path
from torchvision.transforms import v2
import matplotlib.pyplot as plt
import torch

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
scale = 0.5

# load the intrinsic
_K = load_intrinsics(data_dir=data_dir, return_tensor="dict")
K = torch.tensor(
    [
        [_K["fx"] * scale, 0.0, _K["cx"] * scale],
        [0.0, _K["fy"] * scale, _K["cy"] * scale],
        [0.0, 0.0, 1.0],
    ]
)

# load the depth image
_depth_masked = load_depth_masked(data_dir, 0, return_tensor="pt")
_H, _W = _depth_masked.shape
H, W = int(_H * scale), int(_W * scale)

# get the mask
_mask = _depth_masked == 0.0
mask = v2.functional.resize(_mask.unsqueeze(0), size=(H, W)).squeeze(0)
mask = ~mask

# get the new size of the depth image
depth_masked = v2.functional.resize(_depth_masked.unsqueeze(0), size=(H, W)).squeeze(0)

# span the pixel indexes
x = torch.arange(W)
y = torch.arange(H)
idx = torch.stack(torch.meshgrid(y, x), dim=-1).flip(-1)

# get the points in camera coordinates, but with the new resolution
points = torch.concat([idx, depth_masked.unsqueeze(-1)], dim=-1)
points[:, :, 0] *= points[:, :, 2]
points[:, :, 1] *= points[:, :, 2]
out = K.inverse() @ points.permute(2, 0, 1).reshape(3, -1)
out = out.reshape(3, points.shape[0], points.shape[1]).permute(1, 2, 0)

# just save the point of the face
cpoints = out[mask]
np.save("temp/out", cpoints.detach().cpu().numpy())
# [depth_masked] * 3
# depth_masked.shape

In [ ]:
from lib.renderer.camera import depth2camera
from lib.utils.loader import load_depth_masked, load_intrinsics
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
depth = load_depth_masked(data_dir=data_dir, idx=0, return_tensor="pt")
scale = 0.1
K = load_intrinsics(data_dir=data_dir, return_tensor="pt", scale=scale)
points = depth2camera(depth, K, scale=scale)
plt.imshow(points[:, :, 2])
points[:, :, 2].min(), points[:, :, 2].max()

In [ ]:
from lib.renderer.camera import camera2normal
import torch

normals = camera2normal(points.unsqueeze(0))

# now show the image
b_mask = normals.sum(-1) == 0
normals_image = (((normals + 1) / 2) * 255).to(torch.uint8)
normals_image[b_mask, :] = 0
plt.imshow(normals_image[0])
# points.unsqueeze(0).shape

In [ ]:
import torch

# https://stackoverflow.com/questions/34644101/calculate-surface-normals-from-depth-image-using-neighboring-pixels-cross-produc/34644939#34644939
# they have (-dz/dx,-dz/dy,1, however we are in camera space hence we need to calculate the gradient in pixel space, e.g. also the delta x and delta y are in camera space.

# make sure that on the boundary is nothing wrong calculated
points[points.sum(-1) == 0] = torch.nan

H, W, C = points.shape
normals = torch.ones_like(points)
normals *= -1

# we calculate the normal in camera space, hence we also need to normalize with the depth information,
# note that the normal is basically on which direction we have the stepest decent.
x_right = torch.arange(2, W)
x_left = torch.arange(0, W - 2)
normals[:, 1:-1, 0] = (points[:, x_right, 2] - points[:, x_left, 2]) / (
    points[:, x_right, 0] - points[:, x_left, 0]
)

y_right = torch.arange(2, H)
y_left = torch.arange(0, H - 2)
normals[1:-1, :, 1] = (points[y_right, :, 2] - points[y_left, :, 2]) / (
    points[y_right, :, 1] - points[y_left, :, 1]
)

# normalized between [-1, 1]
normals = normals / torch.norm(normals, dim=-1).unsqueeze(-1)
normals = torch.nan_to_num(normals, 0)
normals[:1, :, :] = 0
normals[-1:, :, :] = 0
normals[:, :1, :] = 0
normals[:, -1:, :] = 0
b_mask = normals.sum(-1) == 0


# now show the image
normals_image = (((normals + 1) / 2) * 255).to(torch.uint8)
normals_image[b_mask, :] = 0
plt.imshow(normals_image)

In [ ]:
(points[:, x_right, 2] - points[:, x_left, 2]).max()

In [ ]:
import open3d as o3d

import numpy as np

camera = np.load("temp/out.npy").reshape(-1, 3)
points = camera[camera[:, 2] != 0]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_plotly([pcd])

We can see that we have a point in 3D which is:

[-0.051, -0.042, 0.575] (x, y, z)

The coresponding pixel value is:

[878, 480] (x, y)

How do we get from 3D to 2D screen coordinates?

Input:
fx = 914.415
fy = 914.03
cx = 959.598
cy = 547.202
xyz_camera = [-0.051, -0.042, 0.575] (x, y, z_c)

Output:
uvz_pixel = [878.0, 480.0, 0.575] (u, v, z_c)


In [ ]:
from lib.utils.loader import load_pipnet_landmark_3d
from lib.renderer.camera import load_intrinsics, camera2pixel

flame_landmarks = load_static_landmark_embedding(flame_dir)
lm_idx = flame_landmarks["landmark_indices"]

plt.imshow(color)

lm3d = load_mediapipe_landmark_3d(data_dir, idx=image_idx)
K = load_intrinsics(data_dir=data_dir)
lm = camera2pixel(lm3d, **K)
for point in lm[lm_idx]:
    x, y, z = point.astype(int)
    plt.scatter(x, y, c="red", s=1)  # Drawing a red point for each landmark
plt.show()

# Normals and Points in 3D


In [ ]:
from lib.utils.loader import load_normals_3d, load_points_3d
import open3d as o3d

normals = load_normals_3d(data_dir=data_dir, idx=0)
print(f"{normals.shape=}")
print(normals[:5, :])

points = load_points_3d(data_dir=data_dir, idx=0)
print(f"{points.shape=}")
print(points[:5, :])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_plotly([pcd])

In [ ]:
import open3d as o3d
from lib.utils.loader import load_points_3d
from pathlib import Path

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
points = load_points_3d(data_dir=data_dir, idx=0)
print(f"{points.shape=}")
print(points[:5, :])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_plotly([pcd])

In [ ]:
import numpy as np

path = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/camera/c00_color_extrinsic.txt"
E = np.zeros((4, 4))
E[3, 3] = 1.0
E[:3, :] = np.loadtxt(path).reshape(3, 4)  # extrinsic hence world to camera

# note that the pose is the camera to world, e.g. if flame calls them pose they mean
# that they project from camera to world coordinates, hence the final mesh vertices lives
# in the world coordinate system! This is so important!
# note that this is 4x4
# we need to project the point from camera to world! because the point cloud is in camera
# we can see that because the coordinate system is right-hand where z-axes goes inside and
# y-axes goes down, usually z goes to the camera and y up (see cv2 reference)
pose = np.linalg.inv(E)  # camera to world, hence this is the "pose" they call it that.

points_c_homo = np.zeros((points.shape[0], 4))
points_c_homo[:, 3] = 1.0
points_c_homo[:, :3] = points


points_w_homo = (E @ points_c_homo.T).T

In [ ]:
points_w_homo = (pose[:3, :3] @ points.T).T
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_w_homo)
o3d.visualization.draw_plotly([pcd])

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_w_homo[:, :3])
o3d.visualization.draw_plotly([pcd])